<a href="https://colab.research.google.com/github/AstroBoy1/santander/blob/master/stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
***

In [1]:
!pip install vecstack
!pip install catboost

  Stored in directory: /root/.cache/pip/wheels/35/6d/ca/bce17942bcf7c267b13c97c9c95e2f0ecf0b42160e6074f448
Successfully built vecstack
    100% |████████████████████████████████| 60.1MB 708kB/s 


In [2]:
from google.colab import drive
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn import svm
from collections import Counter
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import time
import statistics
from sklearn.model_selection import GridSearchCV
from xgboost.sklearn import XGBClassifier
from vecstack import stacking
from vecstack import StackingTransformer
import pandas as pd
from glob import glob
import re 
np.random.seed(0) # ensure reproducibility
np.set_printoptions(suppress = True)
from sklearn.datasets import make_classification
from sklearn.metrics import log_loss
# Models
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from keras.wrappers.scikit_learn import KerasClassifier
# Stacking
from vecstack import stacking
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Flatten, Dropout, BatchNormalization, GaussianNoise
from keras import callbacks
import keras.backend as K
from keras.wrappers.scikit_learn import KerasClassifier
import warnings
warnings.simplefilter("ignore")
from keras.callbacks import EarlyStopping, TensorBoard
from mlxtend.feature_selection import ColumnSelector
from sklearn.pipeline import make_pipeline

Using TensorFlow backend.


In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
train_fn = '/content/gdrive/My Drive/santander_data/train.csv'
valid_fn = '/content/gdrive/My Drive/santander_data/test.csv'
pred_fn = '/content/gdrive/My Drive/santander_data/submission12.csv'
train_data_df = pd.read_csv(train_fn)
test_data_df = pd.read_csv(valid_fn)
train_data_x = train_data_df.drop(columns=["ID_code", "target"]).values
train_data_y = train_data_df["target"].values
test_data_x = test_data_df.drop(columns=["ID_code"]).values

# Prepare data

In [0]:
n_classes = 2
length = 1000
X, y = train_data_x[:length], train_data_y[:length]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
save_directory = '/content/gdrive/My Drive/santander_results/'

# LGB, XGB, Catboost params

In [0]:
lgb_params = {
    'bagging_freq': 5,
    'bagging_fraction': 0.335,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.041,
    'learning_rate': 0.0083,
    'max_depth': -1,
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': -1}

xgb_params = {'tree_method': 'hist',
 'objective': 'binary:logistic',
 'eval_metric': 'auc',
 'learning_rate': 0.0936165921314771,
 'max_depth': 2,
 'colsample_bytree': 0.3561271102144279,
 'subsample': 0.8246604621518232,
 'min_child_weight': 53,
 'gamma': 9.943467991283027,
 'silent': 1,
}

catboost_params = {'subsample':0.36, #rawdata 0.5  ×2 0.45 ×3 0.36
                            'loss_function':'Logloss',
                           'random_strength':0,
                           'max_depth':3,
                           'eval_metric':"AUC",
                           'learning_rate':0.02,
                           #'iterations':60000,
                           'iterations':100,
                           #class_weights=[1,2],
                           'bootstrap_type':'Bernoulli',
                           #rsm=0.045,
                            'l2_leaf_reg':0.3,
                           #'task_type':"GPU",
                           'random_seed':432013,
                           'od_type':"Iter",
                           'border_count':128,
                           'logging_level':'Silent'
                           #has_time= True 
                  }

# Wrappers

In [0]:
class WrapLGB(LGBMClassifier):
    """This is template for user-defined class wrapper.
    Use this template to pass any ``fit`` and ``predict`` arguments.
    """
    def fit(self, X, y):
        X_tr, X_val, y_tr, y_val = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)
        return super(WrapLGB, self).fit(X_tr, y_tr, 
                                        early_stopping_rounds=5, 
                                        eval_set=[(X_val, y_val)], 
                                        eval_metric='auc', verbose=1)

    def predict(self, X):
        return super(WrapLGB, self).predict(X, 
               num_iteration=self.best_iteration_)

# NN

In [0]:
def auc(y_true, y_pred):
    """ROC AUC metric for both binary and multiclass classification.
    
    Parameters
    ----------
    y_true : 1d numpy array
        True class labels
    y_pred : 2d numpy array
        Predicted probabilities for each class
    """
    ohe = OneHotEncoder(sparse=False)
    y_true = ohe.fit_transform(y_true.reshape(-1, 1))
    auc_score = roc_auc_score(y_true, y_pred)
    return auc_score

# LOGGER
class Logger(callbacks.Callback):
    def __init__(self, out_path='./', patience=10, lr_patience=3, out_fn='', log_fn=''):
        self.auc = 0
        self.path = out_path
        self.fn = out_fn
        self.patience = patience
        self.lr_patience = lr_patience
        self.no_improve = 0
        self.no_improve_lr = 0

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        cv_pred = self.model.predict(self.validation_data[0], batch_size=1024)
        cv_true = self.validation_data[1]
        auc_val = roc_auc_score(cv_true, cv_pred)
        if self.auc < auc_val:
            self.no_improve = 0
            self.no_improve_lr = 0
            print("Epoch %s - best AUC: %s" % (epoch, round(auc_val, 4)))
            self.auc = auc_val
            self.model.save(self.path + self.fn, overwrite=True)
        else:
            self.no_improve += 1
            self.no_improve_lr += 1
            print("Epoch %s - current AUC: %s" % (epoch, round(auc_val, 4)))
            if self.no_improve >= self.patience:
                self.model.stop_training = True
            if self.no_improve_lr >= self.lr_patience:
                lr = float(K.get_value(self.model.optimizer.lr))
                K.set_value(self.model.optimizer.lr, 0.75*lr)
                print("Setting lr to {}".format(0.75*lr))
                self.no_improve_lr = 0

        return

# MODEL DEF
def dnn():
#     inp = Input(shape=(200, 1))
#     d1 = Dense(16, activation='relu')(inp)
#     fl = Flatten()(d1)
#     preds = Dense(1, activation='sigmoid')(fl)
#     model = Model(inputs=inp, outputs=preds)
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    model = Sequential()
    model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

logger = Logger(patience=10, out_path=save_directory, out_fn='cv_{}.h5')
#nn_params = {'nb_epoch':32, 'batch_size':256, 'callbacks':[logger], 'verbose':1}
nn_params = {'epochs':32, 'batch_size':256, 'verbose':1}

# Stacking Pipeline

In [24]:
# Specify steps of Pipeline

# Same paramaters to models, but different subfeatures by:
# 1. Select a random number for the number of features to select
# 2. Randomly select that many features
num_subsets = 1
subsets = []
features = list(range(200))
number = list(range(1, 201))
pipe_models_1 = []
for num in range(num_subsets):
    num_features = np.random.choice(number, size=1)[0]
    subset = np.random.choice(features, size=num_features, replace=False, p=None)
    subsets.append(subset)
    cl1 = ('gnb' + str(num), make_pipeline(ColumnSelector(cols=tuple(subset)), GaussianNB()))
    cl2 = ('xgb' + str(num), make_pipeline(ColumnSelector(cols=tuple(subset)), XGBClassifier(**xgb_params)))
    cl3 = ('lgbm' + str(num), make_pipeline(ColumnSelector(cols=tuple(subset)), WrapLGB(**lgb_params)))
    cl4 = ('cb' + str(num), make_pipeline(ColumnSelector(cols=tuple(subset)), CatBoostClassifier(**catboost_params)))
    pipe_models_1.append(cl1)
    pipe_models_1.append(cl2)
    pipe_models_1.append(cl3)
    pipe_models_1.append(cl4)
print("Number of level 1 models: ", len(pipe_models_1))
print("Number of subsets: ", len(subsets))
print("Subsets: ", subsets)
# pipe_models_1 = [ 
#     ('gnb', GaussianNB()),
#     #('lg', LogisticRegression(random_state=0)),
#     #('nn', KerasClassifier(build_fn=dnn, epochs=2, batch_size=32, verbose=0, callbacks=[logger])),
#     #('nn', KerasClassifier(build_fn=dnn, epochs=2, batch_size=32, verbose=0)),
#     #('etc', ExtraTreesClassifier(random_state=0, n_jobs=-1, n_estimators=100, max_depth=3)),
#     #('rfc', RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=100, max_depth=3)),
#     ('xgb', XGBClassifier(**xgb_params)),           
#     ('lgbm', LGBMClassifier(**lgb_params)),
#     ('cb', CatBoostClassifier(**catboost_params))
# ]
subset = list(range(0, len(pipe_models_1), 2))
#subset = list(range(0, len(pipe_models_1)))
pipe_models_2 = [ 
    #('gnb', GaussianNB()),
    #('nn', KerasClassifier(build_fn=dnn, epochs=2, batch_size=32, verbose=1)),
    #('etc', ExtraTreesClassifier(random_state=0, n_jobs=-1, n_estimators=100, max_depth=3)),
    #('rfc', RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=100, max_depth=3)),
    ('xgb', make_pipeline(ColumnSelector(cols=tuple(subset)), XGBClassifier(**xgb_params))),           
    ('lgbm', make_pipeline(ColumnSelector(cols=tuple(subset)), LGBMClassifier(**lgb_params))),
    ('cb', make_pipeline(ColumnSelector(cols=tuple(subset)), CatBoostClassifier(**catboost_params)))
]

stack1 = StackingTransformer(pipe_models_1,                   # list of models
                               #X_train, y_train, X_test,   # data
                               regression=False,           # classification task (if you need 
                                                           #     regression - set to True)
                               #mode='oof_pred',            # mode: oof for train set, fit on full 
                               variant='A',                            #     train and predict test set once
                               needs_proba=True,           # predict probabilities (if you need 
                                                           #     class labels - set to False) 
                               ##save_dir='.',               # save result and log in current dir 
                               #save_dir=save_directory,                                 #     (to disable saving - set to None)
                               metric=auc,            # metric: callable
                               n_folds=5,                  # number of folds
                               stratified=True,            # stratified split for folds
                               shuffle=True,               # shuffle the data
                               random_state=0,             # ensure reproducibility
                               verbose=1)                  # print all info

stack2 = StackingTransformer(pipe_models_2,                   # list of models
                               #X_train, y_train, X_test,   # data
                               regression=False,           # classification task (if you need 
                                                           #     regression - set to True)
                               #mode='oof_pred',            # mode: oof for train set, fit on full 
                               variant='A',                            #     train and predict test set once
                               needs_proba=True,           # predict probabilities (if you need 
                                                           #     class labels - set to False) 
                               ##save_dir='.',               # save result and log in current dir 
                               #save_dir=save_directory,                                 #     (to disable saving - set to None)
                               metric=auc,            # metric: callable
                               n_folds=5,                  # number of folds
                               stratified=True,            # stratified split for folds
                               shuffle=True,               # shuffle the data
                               random_state=0,             # ensure reproducibility
                               verbose=1)                  # print all info

Number of level 1 models:  4
Number of subsets:  1
Subsets:  [array([ 16,  31,  95, 154, 125,   5,  53, 123, 114, 146,  96,  14, 118])]


In [0]:
from scipy.stats import rankdata
from sklearn.base import BaseEstimator, TransformerMixin
class Rank(BaseEstimator, TransformerMixin):
    """Define fit and transform for sklearn pipeline"""
    def __init__(self, method='average'):
        self.method = method
    def fit(self, X, y):
        return self
    def transform(self, X):
        return np.apply_along_axis(rankdata, 0, X)
        #return X
stack3 = make_pipeline(Rank(), XGBClassifier(**xgb_params))

steps = [('stack1', stack1),
         ('stack2', stack2),
        ('stack3', stack3)]

# Init Pipeline
pipe = Pipeline(steps)

In [26]:
# Fit
pipe = pipe.fit(X_train, y_train)

task:         [classification]
n_classes:    [2]
metric:       [auc]
variant:      [A]
n_estimators: [4]

estimator  0: [gnb0: Pipeline]
    MEAN:     [0.53753113] + [0.07793492]

estimator  1: [xgb0: Pipeline]
    MEAN:     [0.50000000] + [0.00000000]

estimator  2: [lgbm0: Pipeline]
[1]	valid_0's auc: 0.325397
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's auc: 0.287115
[3]	valid_0's auc: 0.348273
[4]	valid_0's auc: 0.343604
[5]	valid_0's auc: 0.358077
[6]	valid_0's auc: 0.346405
[7]	valid_0's auc: 0.356676
[8]	valid_0's auc: 0.355742
[9]	valid_0's auc: 0.373016
[10]	valid_0's auc: 0.442577
[11]	valid_0's auc: 0.468721
[12]	valid_0's auc: 0.452848
[13]	valid_0's auc: 0.439776
[14]	valid_0's auc: 0.43324
[15]	valid_0's auc: 0.460317
[16]	valid_0's auc: 0.439776
Early stopping, best iteration is:
[11]	valid_0's auc: 0.468721
[1]	valid_0's auc: 0.568627
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's auc: 0.49113
[3]	valid_0's auc: 

# roc on validation data

In [19]:
y_pred = pipe.predict_proba(X_test)
from sklearn.metrics import roc_auc_score

# Final prediction score
# print('Final prediction score: %.8f' % log_loss(y_test, y_pred))
y_pred_final = [elem[1] for elem in y_pred]
roc_auc_score(y_test, y_pred_final)

Transforming...

estimator  0: [gnb0: Pipeline]
    DONE

estimator  1: [xgb0: Pipeline]
    DONE

estimator  2: [lgbm0: Pipeline]
    DONE

estimator  3: [cb0: Pipeline]
    DONE

Transforming...

estimator  0: [xgb: Pipeline]
    DONE

estimator  1: [lgbm: Pipeline]
    DONE

estimator  2: [cb: Pipeline]
    DONE



0.5

# rocauc on training data

In [15]:
y_pred = pipe.predict_proba(train_data_x)
y_pred_final = [elem[1] for elem in y_pred]
roc_auc_score(train_data_y, y_pred_final)

Transforming...

estimator  0: [gnb0: Pipeline]
    DONE

estimator  1: [xgb0: Pipeline]
    DONE

estimator  2: [lgbm0: Pipeline]
    DONE

estimator  3: [cb0: Pipeline]
    DONE

Transforming...

estimator  0: [xgb: Pipeline]
    DONE

estimator  1: [lgbm: Pipeline]
    DONE

estimator  2: [cb: Pipeline]
    DONE



0.5

In [0]:
# Save Pipeline
import joblib
_ = joblib.dump(pipe, save_directory + 'pipe_with_stack.pkl')
# Load Pipeline
pipe_loaded = joblib.load(save_directory + 'pipe_with_stack.pkl')

# Predict on Kaggle Test

In [0]:
kaggle_pred = pipe.predict_proba(test_data_x)

In [0]:
output_df = pd.DataFrame()
output_df["ID_code"] = test_data_df["ID_code"]
pred_final = [elem[1] for elem in kaggle_pred]
output_df["target"] = pred_final
output_df.to_csv(save_directory + "predictions.csv", index=False)